# CityBikes

Send a request to CityBikes for the city of your choice.

In [9]:
# imports

import requests
import json # in vscode, this import is needed to parse the JSON from the api call
import pandas as pd
import numpy as np

# from IPython.display import JSON # for nice JSON (or dict) display in JupyterLab (not currently supported in Jupyter Notebook)

Contact the server via an API call

In [10]:
# set city for investigation - Toronto AKA Bike Share Toronto
city_name = "Toronto, ON"

# get network response for all bike share networks in the world first
response = requests.request('GET', url="https://api.citybik.es/v2/networks")
data_networks = response.json()

# normalize it into a dataframe
df_networks_raw = pd.json_normalize(data_networks['networks'])
df_networks_raw

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,Parco dei Colli di Bergamo,IT,45.722956,9.649230,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,[PickeBike],/v2/networks/pickebike-aubonne,pickebike-aubonne,PickeBike Aubonne,Aubonne,CH,46.494900,6.396300,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
786,"[Mobilitätsakademie AG, Touring Club Switzerla...",/v2/networks/carvelo,carvelo,carvelo eCargo-Bike Sharing,Switzerland,CH,46.982900,8.233100,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/ca...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
787,[PickeBike],/v2/networks/pickebike-basel,pickebike-basel,PickeBike Basel,Basel,CH,47.557900,7.590200,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
788,[PickeBike],/v2/networks/pickebike-fribourg,pickebike-fribourg,PickeBike Fribourg,Fribourg,CH,46.803900,7.150500,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...


790 Cities in lots of countries, I want to find Toronto, Canada

In [11]:
# Country is a two letter symbol so search Canada with CA
df_networks_raw[df_networks_raw['location.country'] == 'CA']

# Toronto is found under 'Toronto, ON' not 'Toronto'

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
70,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-montreal,bixi-montreal,Bixi,"Montréal, QC",CA,45.508693,-73.553928,NaN,https://gbfs.velobixi.com/gbfs/gbfs.json,True,NaN,NaN
72,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-toronto,bixi-toronto,Bike Share Toronto,"Toronto, ON",CA,43.653226,-79.383184,NaN,https://tor.publicbikesystem.net/ube/gbfs/v1/,NaN,NaN,NaN
73,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,"Hamilton, ON",CA,43.256436,-79.869297,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,NaN
199,"[Vanncouver Bike Share Inc., CycleHop LLC, Cit...",/v2/networks/mobibikes,mobibikes,Mobi,Vancouver,CA,49.282700,-123.120700,NaN,https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs....,NaN,NaN,NaN
504,[PBSC Urban Solutions],/v2/networks/avelo-quebec,avelo-quebec,àVélo,Québec,CA,46.813050,-71.216800,NaN,https://quebec.publicbikesystem.net/customer/g...,NaN,NaN,NaN
505,[PBSC Urban Solutions],/v2/networks/acces-velo-saguenay,acces-velo-saguenay,Accès Vélo,Saguenay,CA,48.433333,-71.083333,NaN,https://saguenay.publicbikesystem.net/customer...,NaN,NaN,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).

In [12]:
# we have the href for toronto /v2/networks/bixi-toronto and according to api documentation we make the call by adding fields ?field=list,of,fields

href_toronto = "/v2/networks/bixi-toronto"

response = requests.request('GET', url="https://api.citybik.es/v2/networks/bixi-toronto?fields=stations")
data_stations = response.json()


**Note on CityBike API Call:**
- Current call was made at 3pm on Saturday afternoon, early August
- Warm clear weather, Morning brunch / breakfast travel, bars not open yet

Put your parsed results into a DataFrame.

In [13]:
df_toronto_stations = pd.json_normalize(data_stations['network']['stations'])
df_toronto_stations

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.altitude,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid,extra.post_code
0,1,18,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,2024-08-03T19:09:00.502000Z,Queen St E / Woodward Ave,0.0,0,True,1722711993,18,"[key, transitcard, creditcard, phone]",True,1,1,19,7303,NaN
1,14,1,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,2024-08-03T19:09:00.524000Z,Primrose Ave / Davenport Rd,0.0,0,True,1722711972,1,"[key, transitcard, creditcard, phone]",True,1,1,15,7301,NaN
2,11,12,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,2024-08-03T19:09:00.612000Z,Queen St. E / Rhodes Ave.,0.0,1,True,1722711968,11,"[key, transitcard, creditcard, phone]",True,1,1,23,7309,NaN
3,32,5,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,2024-08-03T19:09:00.689000Z,Bond St / Queen St E,0.0,1,True,1722712044,4,"[key, transitcard, creditcard, phone]",True,1,1,37,7045,NaN
4,32,3,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,2024-08-03T19:09:00.696000Z,Church St / Alexander St,0.0,0,True,1722712001,3,"[key, transitcard, creditcard, phone]",True,1,1,35,7044,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,10,1,9be5f078a1ed47fc11cd3cee45260f63,43.741906,-79.271819,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),2024-08-03T19:09:04.277000Z,Kennedy Rd / Ranstone Gardens,0.0,0,True,1722712023,1,"[key, transitcard, phone]",False,1,1,15,7964,M1K 2E9
822,11,8,4ae37f3bddfb819954a15143d277dbd9,43.736953,-79.247984,Eglinton Ave E / Brimley Rd,2024-08-03T19:09:04.290000Z,Eglinton Ave E / Brimley Rd,0.0,0,True,1722712082,8,"[key, transitcard, phone]",False,1,1,19,7965,M1J 2C6
823,11,7,e7968ab22d9a15db0673f463144428eb,43.659457,-79.382365,College Park South,2024-08-03T19:09:01.655000Z,College Park South,0.0,0,True,1722712069,7,"[key, transitcard, creditcard, phone]",True,1,1,19,7077,NaN
824,11,4,62acc308c0f93ff09d28e06c73afc3ec,43.705875,-79.368006,165 McRae Dr,2024-08-03T19:09:00.995000Z,165 McRae Dr,0.0,0,True,1722712130,4,"[key, transitcard, phone]",False,1,1,15,7779,M4G 1S8


In [14]:
df_toronto_stations.shape

(826, 20)

In [17]:
# saving the df to csv so I can save the data retrieved
csv_file_path = 'bike_stations.csv'
df_toronto_stations.to_csv(csv_file_path, index=False)

In [7]:
# extract the columns of interest into a new df to strip out the irrelevant/extra columns

df_toronto = df_toronto_stations[['latitude', 'longitude', 'free_bikes', 'empty_slots', 'name']]
df_toronto.head(10)

,latitude,longitude,free_bikes,empty_slots,name
0,43.665269,-79.319796,16,3,Queen St E / Woodward Ave
1,43.671420,-79.445947,2,13,Primrose Ave / Davenport Rd
2,43.666224,-79.317693,7,16,Queen St E / Rhodes Ave
3,43.653236,-79.376716,16,21,Bond St / Queen St E
4,43.663722,-79.380288,12,21,Church St / Alexander St
5,43.657763,-79.389165,8,17,University Ave / Gerrard St W (East Side)
6,43.656729,-79.382736,1,22,Edward St / Yonge St
7,43.664467,-79.414783,5,9,Euclid Ave / Bloor St W
8,43.675492,-79.388858,16,7,Yonge St / Aylmer Ave
9,43.674991,-79.396273,5,6,Davenport Rd / Avenue Rd


In [19]:
csv_file_path = 'bike_stations_3pm_sat_clean.csv'
df_toronto.to_csv(csv_file_path, index=False)

**Data Obtained**
- I have saved bike_stations_3pm_sat.csv and bike_stations_3pm_sat_clean to my github repository to save the data for prime time saturday afternoon as I feel the time of day/week does have an impact on bike availability that we will see in the model. I will try this with a less prime time such as monday early afternoon (everyone is at work) or tuesday morning (typically an "in-office" day in Toronto) to see if there's a noticeable difference

**Thoughts on Data viz**
- The citybik.es website visualizes the occupancy of the stations on the front-end website, green with many bikes available, yellow has few bikes available and red has no bikes available. Visually there are a lot more greens in major downtown arteries than in the uptown and fringe areas.
- We should look at an sns heat map of occupancy rate and see if it tracks to major downtown areas. The visualization should be latitude vs. Longitude with the colour of each dot representing the occupancy rate of the bike stations.
- Lat/Long colour viz could be enhanced with size of the points indicating number of amenities returned by the Foursquare API call.
- If we want, we could signpost major arteries in Toronto e.g. Yonge/King, Yonge/Bloor, Bathurst/College as we expect more traffic there than other places generally in Downtown